In [ ]:
import pandas as pd
import numpy as np

def analyse_health(
        df,
        signal,
        gap_threshold,
        lower_limit,
        upper_limit,
        noise_window):
    """
    Docstring for analyse_health
    
    :param df: Data frame containing the data
    :param signal: Name of the column containing the data
    :param gap_threshold: Description
    :param lower_limit: Lower limit of valid values for the signal
    :param upper_limit: Upper limit of valid values for the signal
    :param noise_window: Noise detection rolling window size
    """

    # Calculate the time delta between samples and identify gaps larger than expected
    df["dt"] = pd.to_timedelta(df.index.diff(), errors="coerce")
    df["is_gap"] = df["dt"] > pd.Timedelta(gap_threshold)
    df["segment_id"] = df["is_gap"].cumsum()

    # Create a gaps dataframe
    gaps = df[df["is_gap"]].copy()

    if not gaps.empty:
        # gap_end: where data resumes
        gaps["gap_end"] = gaps.index

        # gap_start: previous timestamp in the original df
        prev_index = df.index.to_series().shift(1)
        gaps["gap_start"] = prev_index.loc[gaps.index]

        # duration and midpoint
        gaps["gap_duration"] = gaps["gap_end"] - gaps["gap_start"]
        gaps["gap_mid"] = gaps["gap_start"] + gaps["gap_duration"] / 2
    else:
        print("No gaps detected above threshold.")

    # Identify out-of-range values
    df["oor"] = (df[signal] < lower_limit) | (df[signal] > upper_limit)
    oor_points = df[df["oor"]]

    # Identify noise/variability within each segment, ignoring gaps
    df["noise"] = (
        df.groupby("segment_id")[signal]
        .rolling(window=noise_window, min_periods=10)
        .std()
        .reset_index(level=0, drop=True)
    )

    # For each gap, mask values inside the interval with NaN so matplotlib won"t draw lines across gaps
    for _, row in gaps.iterrows():
        mask = (df.index >= row["gap_start"]) & (df.index <= row["gap_end"])
        df.loc[mask, signal] = np.nan

    return df, gaps, oor_points

In [ ]:
import matplotlib.pyplot as plt

def chart_data_gaps(
        df,
        signal,
        gaps,
        export_folder_path,
        filename):
    """
    Chart gaps in sesnor data
    
    :param df: Data frame containing the data
    :param signal: Name of the column containing the data
    :param gaps: Data frame containing the gap start/end
    :param export_folder_path: Folder where the chart should be exported
    :param filename: Filename the chart should be exported to
    """
    fig, (ax1, ax2) = plt.subplots(
        2, 1,
        figsize=(14, 8),
        sharex=True,
        gridspec_kw={"height_ratios": [3, 1]}
    )

    # -------------------------------
    # Top: signal with shaded gaps + vertical markers
    # -------------------------------
    ax1.plot(df.index, df[signal], label=signal.capitalize())
    ax1.set_ylabel(signal.capitalize())
    ax1.set_title(f"{signal.capitalize()} with Data Gaps Highlighted")
    ax1.grid(True)

    if not gaps.empty:
        for _, g in gaps.iterrows():
            # Shaded region over the gap
            ax1.axvspan(g["gap_start"], g["gap_end"],
                        color="red", alpha=0.15)

            # Vertical marker at the gap end
            ax1.axvline(g["gap_end"],
                        color="red", alpha=0.5, linestyle="--")

    ax1.legend()

    # -------------------------------
    # Bottom: gap duration bars
    # -------------------------------
    ax2.set_title("Gap Durations")
    ax2.set_ylabel("Minutes")
    ax2.grid(True)

    if not gaps.empty:
        # Convert duration to minutes
        gap_minutes = gaps["gap_duration"].dt.total_seconds() / 60.0

        # Duration bars at gap midpoints
        ax2.bar(gaps["gap_mid"], gap_minutes,
                width=0.01, align="center")  # width is cosmetic here

        # Optional: annotate durations
        for _, g in gaps.iterrows():
            mins = g["gap_duration"].total_seconds() / 60.0
            ax2.text(g["gap_mid"], mins,
                    f"{mins:.1f}m",
                    ha="center", va="bottom", fontsize=8, rotation=90)

    ax2.set_ylim(bottom=0)
    ax2.set_xlabel("Time")

    plt.tight_layout()

    # Export to PNG or PDF, if required
    if export_folder_path and filename:
        export_chart(export_folder_path, filename, "png")

    plt.show()

In [ ]:
import matplotlib.pyplot as plt

def chart_out_of_range(
        df,
        signal,
        oor_points,
        lower_limit,
        upper_limit,
        export_folder_path,
        filename):
    """
    Chart out of range data
    
    :param df: Data frame containing the data
    :param signal: Name of the column containing the data
    :param oor_points: Data frame containing out of range points
    :param lower_limit: Lower limit of valid values for the signal
    :param upper_limit: Upper limit of valid values for the signal
    :param export_folder_path: Folder where the chart should be exported
    :param filename: Filename the chart should be exported to
    """
    fig, ax = plt.subplots(figsize=(14, 5))

    # Plot the normal signal
    ax.plot(df.index, df[signal], label=signal.capitalize(), color="blue")

    # Plot out-of-range values as red circular markers
    ax.scatter(
        oor_points.index,
        oor_points[signal],
        color="red",
        s=40,
        marker="o",
        label="Out-of-range"
    )

    # Draw limit lines
    ax.axhline(lower_limit, color="orange", linestyle="--", label=f"Lower limit ({lower_limit})")
    ax.axhline(upper_limit, color="orange", linestyle="--", label=f"Upper limit ({upper_limit})")

    ax.set_title(f"{signal.capitalize()} with Out-of-Range Values Highlighted")
    ax.set_ylabel(signal.capitalize())
    ax.grid(True)
    ax.legend()

    # Export to PNG or PDF, if required
    if export_folder_path and filename:
        export_chart(export_folder_path, filename, "png")

    plt.show()

In [ ]:
import matplotlib.pyplot as plt

def chart_sensor_noise(df, signal, export_folder_path, filename):
    """
    Chart sensor noise
    
    :param df: Data frame containing the data
    :param signal: Name of the column containing the data
    :param export_folder_path: Folder where the chart should be exported
    :param filename: Filename the chart should be exported to
    """
    fig, ax = plt.subplots(figsize=(14,4))

    ax.plot(df.index, df["noise"], label="Noise (STD)", color="purple")
    ax.set_title(f"{signal.capitalize()} Sensor Noise")
    ax.set_ylabel("Rolling Standard Deviation")
    ax.grid(True)
    ax.legend()

    # Export to PNG or PDF, if required
    if export_folder_path and filename:
        export_chart(export_folder_path, filename, "png")

    plt.show()